# Telco Churn — Notebook n°5
## Gradient Boosting (challenger) vs Logistic Regression (finale actuelle)

🎯 Objectif :
- Tester un modèle Gradient Boosting avec le même pipeline (preprocessor + modèle)
- Comparer à la Logistic Regression (ROC-AUC + recall churn + seuil)
- Décider si on remplace ou si on garde la Logistic (impact métier : rétention)


In [1]:
# Imports & settings

import numpy as np
import pandas as pd

from pathlib import Path
import joblib

from sklearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import (
    roc_auc_score,
    classification_report,
    confusion_matrix,
    precision_recall_curve,
    roc_curve
)


In [6]:
# Paths & load data

PATH_PREPROCESSOR = Path("../../models/preprocessor.joblib")  # ex: notebooks/models/...
PATH_LOGISTIC_PIPELINE = Path("../notebooks/models/logreg_pipeline.joblib")


DATA_PATH = Path("../../data/processed")  # adapte si besoin

X_train = joblib.load(DATA_PATH / "X_train.joblib")
X_test  = joblib.load(DATA_PATH / "X_test.joblib")
y_train = joblib.load(DATA_PATH / "y_train.joblib")
y_test  = joblib.load(DATA_PATH / "y_test.joblib")

print("✅ Train / test datasets loaded")

preprocessor = joblib.load(PATH_PREPROCESSOR)
print("✅ Preprocessor loaded:", type(preprocessor))


✅ Train / test datasets loaded
✅ Preprocessor loaded: <class 'sklearn.compose._column_transformer.ColumnTransformer'>


In [7]:
# Sanity check - load train/test data

print("X_train:", X_train.shape, "| X_test:", X_test.shape)
print("y_train:", y_train.shape, "| y_test:", y_test.shape)

print("\nChurn rate train:", y_train.mean())
print("Churn rate test :", y_test.mean())

assert X_train.shape[0] == y_train.shape[0]
assert X_test.shape[0] == y_test.shape[0]


X_train: (5634, 21) | X_test: (1409, 21)
y_train: (5634,) | y_test: (1409,)

Churn rate train: 0.2653532126375577
Churn rate test : 0.2654364797728886


In [8]:
# Evaluating function

def evaluate_model(y_true, y_proba, threshold=0.5, label="model"):
    y_pred = (y_proba >= threshold).astype(int)

    auc = roc_auc_score(y_true, y_proba)
    cm = confusion_matrix(y_true, y_pred)

    print(f"\n📌 {label} — Threshold = {threshold:.2f}")
    print(f"ROC-AUC: {auc:.3f}")
    print("\nClassification report:")
    print(classification_report(y_true, y_pred, digits=3))
    print("Confusion matrix:")
    print(cm)

    tn, fp, fn, tp = cm.ravel()
    recall_churn = tp / (tp + fn) if (tp + fn) > 0 else 0.0

    return {"label": label, "threshold": threshold, "roc_auc": auc, "recall_churn": recall_churn, "cm": cm}


In [9]:
# Pipeline with Gradient Boosting

gb_clf = GradientBoostingClassifier(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=3,
    random_state=42
)

gb_pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("model", gb_clf)
])

gb_pipeline


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['SeniorCitizen', 'tenure',
                                                   'MonthlyCharges',
                                                   'TotalCharges',
                                                   'customer_value',
                                                   'high_monthly_charges']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   One...ore',
                                                                                 sparse_output=False))]),
                                                  ['gender', 'Partner',
                                                   'Dependents', 'PhoneService',
                                                   'MultipleLines',
                                                   'InternetService',
                                                   'OnlineSecurity',
                                                   'OnlineBackup',
                                                   'DeviceProtection',
                                                   'TechSupport', 'StreamingTV',
                                                   'StreamingMovies',
                                                   'Contract',
                                                   'PaperlessBilling',
                                                   'PaymentMethod'])])),
                ('model',
                 GradientBoostingClassifier(learning_rate=0.05,
                                            n_estimators=300,
                                            random_state=42))])

In [10]:
# Train Gradient Boosting model

gb_pipeline.fit(X_train, y_train)
print("✅ Gradient Boosting trained.")


✅ Gradient Boosting trained.


In [11]:
# Evaluate on test set

gb_proba = gb_pipeline.predict_proba(X_test)[:, 1]

res_gb_050 = evaluate_model(y_test, gb_proba, threshold=0.50, label="GradientBoosting")
res_gb_040 = evaluate_model(y_test, gb_proba, threshold=0.40, label="GradientBoosting")



📌 GradientBoosting — Threshold = 0.50
ROC-AUC: 0.842

Classification report:
              precision    recall  f1-score   support

           0      0.838     0.909     0.872      1035
           1      0.671     0.513     0.582       374

    accuracy                          0.804      1409
   macro avg      0.755     0.711     0.727      1409
weighted avg      0.794     0.804     0.795      1409

Confusion matrix:
[[941  94]
 [182 192]]

📌 GradientBoosting — Threshold = 0.40
ROC-AUC: 0.842

Classification report:
              precision    recall  f1-score   support

           0      0.864     0.832     0.848      1035
           1      0.579     0.639     0.607       374

    accuracy                          0.781      1409
   macro avg      0.722     0.735     0.728      1409
weighted avg      0.789     0.781     0.784      1409

Confusion matrix:
[[861 174]
 [135 239]]


In [12]:
# Find optimal threshold for recall

precision, recall, thresholds = precision_recall_curve(y_test, gb_proba)

f1_scores = 2 * (precision * recall) / (precision + recall + 1e-9)
best_idx = int(np.argmax(f1_scores))
best_threshold = float(thresholds[best_idx]) if best_idx < len(thresholds) else 0.5

print("📌 Best threshold (max F1):", round(best_threshold, 4))
print("Precision:", round(float(precision[best_idx]), 4), "| Recall:", round(float(recall[best_idx]), 4))


📌 Best threshold (max F1): 0.3601
Precision: 0.5622 | Recall: 0.7005


In [13]:
# Evaluate at best threshold

res_gb_best = evaluate_model(y_test, gb_proba, threshold=best_threshold, label="GradientBoosting (best F1)")



📌 GradientBoosting (best F1) — Threshold = 0.36
ROC-AUC: 0.842

Classification report:
              precision    recall  f1-score   support

           0      0.881     0.803     0.840      1035
           1      0.562     0.701     0.624       374

    accuracy                          0.776      1409
   macro avg      0.722     0.752     0.732      1409
weighted avg      0.797     0.776     0.783      1409

Confusion matrix:
[[831 204]
 [112 262]]


In [17]:
# Compare with Logistic Regression

PATH_LOGISTIC_PIPELINE = Path("../models/logreg_baseline_pipeline.joblib")  # adapter si besoin

rows = []

if PATH_LOGISTIC_PIPELINE.exists():
    logreg_pipeline = joblib.load(PATH_LOGISTIC_PIPELINE)
    lr_proba = logreg_pipeline.predict_proba(X_test)[:, 1]
    res_lr_040 = evaluate_model(y_test, lr_proba, threshold=0.40, label="LogisticRegression")

    rows.append({
        "Model": "Logistic Regression",
        "ROC-AUC": res_lr_040["roc_auc"],
        "Recall churn": res_lr_040["recall_churn"],
        "Threshold": res_lr_040["threshold"]
    })
else:
    print("⚠️ Logistic pipeline not found → comparaison automatique impossible.")

# Gradient Boosting (seuil 0.40 = comparable au NB3)
rows.append({
    "Model": "Gradient Boosting",
    "ROC-AUC": res_gb_040["roc_auc"],
    "Recall churn": res_gb_040["recall_churn"],
    "Threshold": res_gb_040["threshold"]
})

compare_df = pd.DataFrame(rows).sort_values(by="Recall churn", ascending=False)
compare_df



📌 LogisticRegression — Threshold = 0.40
ROC-AUC: 0.841

Classification report:
              precision    recall  f1-score   support

           0      0.929     0.641     0.758      1035
           1      0.465     0.864     0.604       374

    accuracy                          0.700      1409
   macro avg      0.697     0.752     0.681      1409
weighted avg      0.805     0.700     0.717      1409

Confusion matrix:
[[663 372]
 [ 51 323]]


,Model,ROC-AUC,Recall churn,Threshold
0,Logistic Regression,0.841368,0.863636,0.4
1,Gradient Boosting,0.841510,0.639037,0.4


## Décision finale

Bien que le Gradient Boosting obtienne un ROC-AUC comparable à la Logistic Regression,
il présente un recall churn significativement inférieur (0.64 vs 0.86).

Dans un contexte de rétention client, l’objectif prioritaire est de minimiser les faux négatifs
(clients churners non détectés).

La Logistic Regression est donc retenue comme modèle final :
- meilleur recall churn
- performance globale équivalente
- modèle plus simple et interprétable
